In [2]:
from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "python")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [12]:
print(raw_datasets['train'][90]['whole_func_string'])

def findspans(self, type,set=None):
        """Find span annotation of the specified type that include this word"""
        if issubclass(type, AbstractAnnotationLayer):
            layerclass = type
        else:
            layerclass = ANNOTATIONTYPE2LAYERCLASS[type.ANNOTATIONTYPE]
        e = self
        while True:
            if not e.parent: break
            e = e.parent
            for layer in e.select(layerclass,set,False):
                for e2 in layer:
                    if isinstance(e2, AbstractSpanAnnotation):
                        if self in e2.wrefs():
                            yield e2


In [21]:
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]
training_corpus = get_training_corpus()

In [13]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [19]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''
print(old_tokenizer.tokenize(example))

['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']


In [22]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [25]:
tokens = tokenizer.tokenize(example)
print(tokens)

['def', 'Ġadd', '_', 'numbers', '(', 'a', ',', 'Ġb', '):', 'ĊĠĠĠ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`."""', 'ĊĠĠĠ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']


In [35]:
import requests

url = "https://72e5-35-234-29-5.ngrok.io/execute_code"
code = "print(2 + 2)"

def get_response(url,code):
    params = {"code": code}
    response = requests.post(url, params=params)

    print(response.json())


4



In [36]:
from IPython.core.magic import register_cell_magic
import requests

@register_cell_magic
def get_response(line, cell):
    url = line.strip()
    params = {"code": cell}
    response = requests.post(url, params=params)

    return response.json()


In [37]:
%%get_response https://e1be-35-234-29-5.ngrok.io/execute_code
print(2 + 2)


'4\n'

In [38]:
def execute_code(code: str):
    try:
        global_vars = {}
        local_vars = {}
        exec(code, global_vars, local_vars)
        
        # Capture the output of the executed code
        captured_output = local_vars.get('__builtins__', {}).get('print', lambda *args: None)
        
        output = []
        def custom_print(*args, **kwargs):
            output.append(' '.join(map(str, args)))
            captured_output(*args, **kwargs)
        
        local_vars['__builtins__']['print'] = custom_print
        
        # Execute the code
        exec(code, global_vars, local_vars)
        
        return {'output': '\n'.join(output)}
    except Exception as e:
        return {'error': str(e)}

In [40]:
execute_code('2+2')

{'error': "'__builtins__'"}

In [41]:
local_vars.get('__builtins__', {}).get('print', lambda *args: None)

NameError: name 'local_vars' is not defined

In [43]:
# !pip install fastapi nest-asyncio pyngrok uvicorn
# !pip install datasets

# from fastapi import FastAPI
# from fastapi.middleware.cors import CORSMiddleware
# import subprocess

# app = FastAPI()

# app.add_middleware(
#     CORSMiddleware,
#     allow_origins=['*'],
#     allow_credentials=True,
#     allow_methods=['*'],
#     allow_headers=['*'],
# )

# @app.get('/')
# async def root():
#     return {'hello': 'world'}

# @app.post('/execute_code')
# async def execute_code(code: str):
#     try:
#       result = subprocess.check_output(["python", "-c", code], stderr=subprocess.STDOUT, text=True)
#       return result.strip()
#       # return {'output': result}
#     except subprocess.CalledProcessError as e:
#       print('error')
#       return {'error': e.output}


# import nest_asyncio
# from pyngrok import ngrok
# import uvicorn

# ngrok_tunnel = ngrok.connect(8000)
# print('Public URL:', ngrok_tunnel.public_url)
# nest_asyncio.apply()
# uvicorn.run(app, port=8000)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Traceback (most recent call last):
  File "/Users/pranavjha/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/pranavjha/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/pranavjha/anaconda3/lib/python3.10/site-packages/pip/__main__.py", line 8, in <module>
    if sys.path[0] in ("", os.getcwd()):
FileNotFoundError: [Errno 2] No such file or directory


In [ ]:
/kaggle/working/
